# Imports

In [17]:
import datetime
import json

##  Releasy development version

In [18]:
import os
import sys

releasy_module = os.path.abspath(os.path.join('..','..','..','dev','releasy'))
if releasy_module not in sys.path:
    sys.path.insert(0, releasy_module)

In [19]:
for mod in sorted(sys.modules.keys()):
    if mod.startswith("releasy"):
        del sys.modules[mod]

In [20]:
from releasy.miner.vcs.miner import Miner
from releasy.miner.vcs.git import GitVcs

In [21]:
from releasy.miner.vcs import miner as releasy_miner
from releasy.miner.vcs import git as releasy_git

In [22]:
import importlib
importlib.reload(releasy_miner)
importlib.reload(releasy_git)

<module 'releasy.miner.vcs.git' from '/home/felipecrp/dev/releasy/releasy/miner/vcs/git.py'>

## Data Analysis

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import wilcoxon
from scipy.stats import ranksums

In [24]:
# %matplotlib inline
%matplotlib notebook
pd.options.display.max_rows = 1000

# Variables

In [25]:
repo_path = os.path.join('..','..','..','repos')

In [26]:
projects = pd.read_pickle("projects.zip")
projects["data"] = None

In [27]:
# projects.loc[projects.index.isin(bkp.index), "data"] = bkp.data
# projects = projects[projects.index == "git/git"]

# Process Repos

In [ ]:
count = 1
for name, project in projects[(projects.data.isnull())].iterrows():
    print(f"{datetime.datetime.now()} - {count:3} - Processing {name}")
    project_group, project_name = name.split("/") 
    path = os.path.join(repo_path, project_group, f"{project_name}.git")
    projects.loc[name, "path"] = path    
    
    params = {}
    miner = releasy_miner.Miner(name=name,vcs=releasy_git.GitVcs(path), **params)
    project = miner.mine_commits()
    projects.loc[name, "data"] = project
    count += 1
print(f"{datetime.datetime.now()} - Ended")

2019-11-24 00:40:26.008224 -   1 - Processing freeCodeCamp/freeCodeCamp
2019-11-24 00:40:26.012081 -   2 - Processing vuejs/vue
2019-11-24 00:40:26.576361 -   3 - Processing facebook/react
2019-11-24 00:40:29.793343 -   4 - Processing twbs/bootstrap
2019-11-24 00:40:36.043708 -   5 - Processing facebook/react-native
2019-11-24 00:40:40.400982 -   6 - Processing facebook/create-react-app
2019-11-24 00:40:40.565624 -   7 - Processing axios/axios
2019-11-24 00:40:40.603775 -   8 - Processing nodejs/node
2019-11-24 00:41:35.674016 -   9 - Processing FortAwesome/Font-Awesome
2019-11-24 00:41:35.732818 -  10 - Processing angular/angular.js
2019-11-24 00:41:37.831409 -  11 - Processing microsoft/vscode
2019-11-24 00:44:59.369035 -  12 - Processing microsoft/TypeScript
2019-11-24 00:45:51.116359 -  13 - Processing angular/angular
2019-11-24 00:45:55.001527 -  14 - Processing ant-design/ant-design
2019-11-24 00:45:57.646349 -  15 - Processing reduxjs/redux
2019-11-24 00:45:57.816560 -  16 - Pro

In [92]:
projects.loc[projects.data == 1, "data"] = None

In [143]:
projects.shape

(99, 8)

## Check tags

In [94]:
# bkp = projects.copy()

In [98]:
# projects = bkp[projects.data.notnull()].copy()

In [99]:
projects.shape

(94, 8)

In [145]:
projects["tags"] = projects["data"].apply(lambda project: len(project.tags))
projects["releases"] = 0
projects["percent_releases"] = 0
projects["releases"] = projects["data"].apply(lambda project: len(project.releases))
projects.loc[projects["tags"] > 0,"percent_releases"] = projects["releases"]/projects["tags"]

## Promise 1: Projects often use tags to represent software releases

In [146]:
projects[projects.releases > 0].shape[0] / projects.shape[0]

0.9292929292929293

## Peril 1: Projects do not use tags

In [147]:
projects[projects.releases == 0].shape[0] / projects.shape[0]

0.0707070707070707

In [149]:
list(projects[projects.tags == 0].index)

['freeCodeCamp/freeCodeCamp',
 'mxgmn/WaveFunctionCollapse',
 'pjreddie/darknet',
 'huginn/huginn',
 'freeCodeCamp/devdocs']

In [148]:
list(projects[projects.releases == 0].index)

['freeCodeCamp/freeCodeCamp',
 'mxgmn/WaveFunctionCollapse',
 'curl/curl',
 'pjreddie/darknet',
 'huginn/huginn',
 'freeCodeCamp/devdocs',
 'ruby/ruby']

In [107]:
projects[projects.percent_releases < 1][["tags","releases","percent_releases"]]

,tags,releases,percent_releases
name,,,
freeCodeCamp/freeCodeCamp,0,0,0.000000
d3/d3,260,259,0.996154
facebook/react-native,307,305,0.993485
facebook/create-react-app,324,323,0.996914
nodejs/node,543,540,0.994475
microsoft/vscode,142,137,0.964789
microsoft/TypeScript,110,109,0.990909
angular/angular,311,309,0.993569
ant-design/ant-design,285,284,0.996491


In [ ]:
project_df["created_at"] = pd.to_datetime(project_df["created_at"])

In [ ]:
project_df["n_tags"] = project_df["data"].apply(lambda project: len(project.tags))
project_df["n_releases"] = 0
project_df["p_releases"] = 0
project_df["n_releases"] = project_df["data"].apply(lambda project: len(project.releases))
project_df.loc[project_df["n_tags"] > 0,"p_releases"] = project_df["n_releases"]/project_df["n_tags"]

In [ ]:
project_df.loc[(project_df.n_releases < 5), "discarded_by"] = "few releases"
project_df.loc[(project_df.p_releases < 0.85), "discarded_by"] = "nom semantic"
project_df.loc[(project_df.created_at > "2018-10-01"), "discarded_by"] = "too young"
project_df.loc[(project_df.stars < 1000), "discarded_by"] = "few stars"

In [ ]:
project_df[(project_df.created_at > "2018-10-01")]

In [ ]:
summary = pd.DataFrame()
summary["n_projects"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].count()
summary["older_project_birth"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["created_at"].min()
summary["younger_project_birth"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["created_at"].max()
summary["min_stars"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["stars"].min()
summary["max_stars"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["stars"].max()
summary["min_releases"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].min()
summary["max_releases"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].max()
summary["mean_releases"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].mean().round(0)
summary["std_deviation_releases"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].std().round(0)
summary["total_releases"] = project_df[(project_df.discarded_by.isnull())].groupby(["language"])["n_releases"].sum()
summary

In [ ]:
summary.total_releases.sum()

In [385]:
project_df[(project_df.discarded_by.notnull()) & (project_df.language == "Go")].sort_values(by=["n_releases"], ascending=False)

,name,language,created_at,stars,p_releases,n_releases,n_tags,percent_releases,prefixes,data,url,git_url,discarded_by
162,go-martini/martini,Go,2013-10-30 02:34:07,10719,1.0,3,3,NaN,NaN,go-martini/martini,https://api.github.com/repos/go-martini/martini,git://github.com/go-martini/martini.git,few releases
164,gocolly/colly,Go,2017-09-29 14:08:49,9072,1.0,3,3,NaN,NaN,gocolly/colly,https://api.github.com/repos/gocolly/colly,git://github.com/gocolly/colly.git,few releases
152,avelino/awesome-go,Go,2014-07-06 13:42:15,48855,0.0,0,0,NaN,NaN,avelino/awesome-go,https://api.github.com/repos/avelino/awesome-go,git://github.com/avelino/awesome-go.git,nom semantic
167,golang/groupcache,Go,2013-07-22 21:55:07,7896,0.0,0,0,NaN,NaN,golang/groupcache,https://api.github.com/repos/golang/groupcache,git://github.com/golang/groupcache.git,nom semantic
168,andlabs/ui,Go,2014-02-17 23:44:00,7153,0.0,0,1,NaN,NaN,andlabs/ui,https://api.github.com/repos/andlabs/ui,git://github.com/andlabs/ui.git,nom semantic


In [282]:
project_df[(project_df.n_tags < 5)].sort_values(by=["n_releases"], ascending=False)

,name,language,created_at,stars,p_releases,n_releases,n_tags,percent_releases,prefixes,data,url,git_url,discarded_by
41,NationalSecurityAgency/ghidra,Java,2019-03-01 03:27:48,17883,1.0,4,4,NaN,NaN,NationalSecurityAgency/ghidra,https://api.github.com/repos/NationalSecurityA...,git://github.com/NationalSecurityAgency/ghidra...,few tags
81,doctrine/lexer,PHP,2013-01-12 18:58:26,8014,1.0,4,4,NaN,NaN,doctrine/lexer,https://api.github.com/repos/doctrine/lexer,git://github.com/doctrine/lexer.git,few tags
165,gocolly/colly,Go,2017-09-29 14:08:49,9073,1.0,3,3,NaN,NaN,gocolly/colly,https://api.github.com/repos/gocolly/colly,git://github.com/gocolly/colly.git,few tags
163,go-martini/martini,Go,2013-10-30 02:34:07,10719,1.0,3,3,NaN,NaN,go-martini/martini,https://api.github.com/repos/go-martini/martini,git://github.com/go-martini/martini.git,few tags
130,lpereira/lwan,C,2012-01-28 00:48:12,5064,1.0,2,2,NaN,NaN,lpereira/lwan,https://api.github.com/repos/lpereira/lwan,git://github.com/lpereira/lwan.git,few tags
139,imathis/octopress,Ruby,2009-10-18 21:41:32,9503,1.0,1,1,NaN,NaN,imathis/octopress,https://api.github.com/repos/imathis/octopress,git://github.com/imathis/octopress.git,few tags
131,zserge/webview,C,2017-08-19 08:26:00,4901,1.0,1,1,NaN,NaN,zserge/webview,https://api.github.com/repos/zserge/webview,git://github.com/zserge/webview.git,few tags
125,liuliu/ccv,C,2010-09-15 15:59:47,6557,0.0,0,0,NaN,NaN,liuliu/ccv,https://api.github.com/repos/liuliu/ccv,git://github.com/liuliu/ccv.git,nom semantic
119,nothings/stb,C,2014-05-25 16:51:23,10960,0.0,0,0,NaN,NaN,nothings/stb,https://api.github.com/repos/nothings/stb,git://github.com/nothings/stb.git,nom semantic
47,vinta/awesome-python,Python,2014-06-27 21:00:06,74680,0.0,0,0,NaN,NaN,vinta/awesome-python,https://api.github.com/repos/vinta/awesome-python,git://github.com/vinta/awesome-python.git,nom semantic


In [73]:
project_df[project_df.p_releases > 0.85].groupby("language").count()

,name,stars,p_releases,n_releases,n_tags,percent_releases,prefixes,data,url,git_url,discarded_by
language,,,,,,,,,,,
C,15,15,15,15,15,15,0,15,15,15,12
C#,15,15,15,15,15,15,0,15,15,15,14
C++,15,15,15,15,15,15,0,15,15,15,14
Go,14,14,14,14,14,14,0,14,14,14,10
Java,15,15,15,15,15,15,3,15,15,15,14
JavaScript,16,16,16,16,16,16,5,16,16,16,16
PHP,15,15,15,15,15,15,0,15,15,15,14
Python,15,15,15,15,15,15,0,15,15,15,13
Ruby,15,15,15,15,15,15,0,15,15,15,14


In [56]:
project_df[(project_df.n_tags < 10)].head(20)

,name,language,stars,p_releases,n_releases,n_tags,percent_releases,prefixes,data,url,git_url,discarded_by
41,NationalSecurityAgency/ghidra,Java,17859,1.000000,4,4,100.00,NaN,NationalSecurityAgency/ghidra,https://api.github.com/repos/NationalSecurityA...,git://github.com/NationalSecurityAgency/ghidra...,NaN
46,vinta/awesome-python,Python,74541,0.000000,0,0,0.00,NaN,vinta/awesome-python,https://api.github.com/repos/vinta/awesome-python,git://github.com/vinta/awesome-python.git,NaN
49,josephmisiti/awesome-machine-learning,Python,42243,0.000000,0,0,0.00,NaN,josephmisiti/awesome-machine-learning,https://api.github.com/repos/josephmisiti/awes...,git://github.com/josephmisiti/awesome-machine-...,NaN
55,google/python-fire,Python,15522,1.000000,6,6,100.00,NaN,google/python-fire,https://api.github.com/repos/google/python-fire,git://github.com/google/python-fire.git,NaN
59,tflearn/tflearn,Python,9293,1.000000,7,7,100.00,NaN,tflearn/tflearn,https://api.github.com/repos/tflearn/tflearn,git://github.com/tflearn/tflearn.git,NaN
77,doctrine/inflector,PHP,8187,1.000000,5,5,100.00,NaN,doctrine/inflector,https://api.github.com/repos/doctrine/inflector,git://github.com/doctrine/inflector.git,NaN
93,apache/incubator-brpc,C++,9268,1.000000,6,6,100.00,NaN,apache/incubator-brpc,https://api.github.com/repos/apache/incubator-...,git://github.com/apache/incubator-brpc.git,NaN
96,thangchung/awesome-dotnet-core,C#,10284,0.000000,0,0,0.00,NaN,thangchung/awesome-dotnet-core,https://api.github.com/repos/thangchung/awesom...,git://github.com/thangchung/awesome-dotnet-cor...,NaN
99,AvaloniaUI/Avalonia,C#,7394,1.000000,8,8,100.00,NaN,AvaloniaUI/Avalonia,https://api.github.com/repos/AvaloniaUI/Avalonia,git://github.com/AvaloniaUI/Avalonia.git,NaN
111,vurtun/nuklear,C,13034,0.000000,0,0,0.00,NaN,vurtun/nuklear,https://api.github.com/repos/vurtun/nuklear,git://github.com/vurtun/nuklear.git,NaN


In [57]:
project_df[(project_df.p_releases < 0.85) & (project_df.discarded_by.isnull()) & (project_df.n_tags > 0)].head(20)

,name,language,stars,p_releases,n_releases,n_tags,percent_releases,prefixes,data,url,git_url,discarded_by
80,BVLC/caffe,C++,29282,0.428571,6,14,42.86,NaN,BVLC/caffe,https://api.github.com/repos/BVLC/caffe,git://github.com/BVLC/caffe.git,NaN
110,curl/curl,C,15248,0.000000,0,184,0.00,NaN,curl/curl,https://api.github.com/repos/curl/curl,git://github.com/curl/curl.git,NaN
112,openssl/openssl,C,11540,0.000000,0,299,0.00,NaN,openssl/openssl,https://api.github.com/repos/openssl/openssl,git://github.com/openssl/openssl.git,NaN
115,andlabs/libui,C,8994,0.428571,3,7,42.86,NaN,andlabs/libui,https://api.github.com/repos/andlabs/libui,git://github.com/andlabs/libui.git,NaN
132,rapid7/metasploit-framework,Ruby,18557,0.591241,324,548,59.12,NaN,rapid7/metasploit-framework,https://api.github.com/repos/rapid7/metasploit...,git://github.com/rapid7/metasploit-framework.git,NaN


In [43]:
for name,project_data in projects_data.items():
    if "discarded_by" in project_data:
        continue
    project = project_data["data"]
    tags = project.tags
    nom_release_tags = [tag for tag in project.tags if not tag.release or tag.name.startswith("zone") or tag.name.startswith("ngcontainer")]
    if len(tags) > 0:
        percent_releases = round(100*(1-len(nom_release_tags)/len(tags)),2)
    else:
        percent_releases = 0
    project_data["percent_releases"] = percent_releases
    
    prefixes = []
    for release in project.releases:
        if release.prefix not in prefixes:
            prefixes.append(release.prefix)

    print(f"{project.name:30} {percent_releases:10} {len(project.releases):10}/{len(project.tags)}")
    print("  Prefixes:")
    for prefix in prefixes:
         print(f"    - {str(prefix):30}")

    print("  Non releases:")            
    for tag in tags:
        if not tag.release:
           print(f"    - {tag.name}")
#        print(f"-- {tag.name:30} {tag.release != None}")
    print("---")





vuejs/vue                           100.0        248/248
  Prefixes:
    -                               
    - v                             
  Non releases:
---
facebook/react                      100.0        116/116
  Prefixes:
    - v                             
    -                               
  Non releases:
---
twbs/bootstrap                      100.0         55/55
  Prefixes:
    - v                             
  Non releases:
---
facebook/react-native               99.35        305/307
  Prefixes:
    - v                             
    -                               
  Non releases:
    - docusaurus-A
    - latest
---
jquery/jquery                       100.0        148/148
  Prefixes:
    -                               
  Non releases:
---
hakimel/reveal.js                   100.0         25/25
  Prefixes:
    -                               
  Non releases:
---
socketio/socket.io                  100.0        128/128
  Prefixes:
    -                             

In [ ]:
f_releases = []
index = 0
last_project = None
for name,project_data in projects_data.items():
    if "percent_releases" in project_data and project_data["percent_releases"] > 90:
        project = project_data["data"]
        feature_releases = project.get_releases(skip_patches=True, skip_pre=True)
        feature_releases = sorted(feature_releases, key=lambda r: r.version)
        last_release = None
        for release in feature_releases:
            if release.patches:
                last_patch = release.patches[-1]
                maintenance_length = last_patch.time - release.time
                num_patches = len(release.patches)
            else:
                maintenance_length = pd.to_timedelta(0)
                last_patch = None
                num_patches = 0
            if release.pre_releases:
                first_pre_release = release.pre_releases[0]
                stage_length = release.time - first_pre_release.time 
                num_pre_releases = len(release.pre_releases)
            else:
                first_pre_release = None
                stage_length = pd.to_timedelta(0)
                num_pre_releases = 0

            if last_release and last_release.major != release.major:
                f_releases[index-1]["is_last_minor"] = True
            last_release = release
            
            if last_project and last_project != name:
                f_releases.pop()
                index -= 1
            last_project = name
            index += 1

            f_releases.append({
                "project": project.name,
                "release": release.name,
                "version": release.version,
                "is_last_minor": False,
                "time": pd.to_datetime(release.time, utc=True),
                "developtment_length": release.length,
                "last_patch": last_patch, 
                "last_patch_time": pd.to_datetime(last_patch.time, utc=True) if last_patch else None, 
                "maintenance_length": maintenance_length,
                "n_patches": num_patches,
                
                #"num_pre_releases": num_pre_releases,
                #"num_patches": num_patches,
                #"first_pre_release": first_pre_release, 
                #"stage_length": stage_length
            })

f_releases = pd.DataFrame(f_releases)    
f_releases = f_releases.sort_values(by=["project", "version"])

f_releases["maintenance_secs"] = f_releases["maintenance_length"].dt.total_seconds()
f_releases["time"] = f_releases["time"].dt.tz_convert(None)
f_releases["last_patch_time"] = f_releases["last_patch_time"].dt.tz_convert(None)

# remove releases sem manutenção
# f_releases = f_releases[f_releases.maintenance_length > pd.to_timedelta(0)].copy()

f_releases.to_excel("feature_releases_ds.xlsx")
f_releases.head(200)

In [ ]:
fig = plt.figure()                                                                                                                                                                                                                                                             
ax = fig.add_subplot(111)

f_releases.boxplot("maintenance_length", by="is_last_minor", ax=ax)

def timeTicks(x, pos):
    return str(pd.to_timedelta(x))
    #d = datetime.timedelta(seconds=x)
    #return str(d)
    #return x / 60 / 60 / 24
formatter = matplotlib.ticker.FuncFormatter(timeTicks)                                                                                                                                                                                                                         
ax.yaxis.set_major_formatter(formatter)

plt.suptitle("")
plt.tight_layout()

## Statistical Test

## Analise pareada por Projeto


In [ ]:
gr = f_releases.groupby(["project","is_last_minor"]).mean()

In [ ]:
gr.shape

In [ ]:
(gr.groupby(level=[0]).size() == 2)

In [ ]:
# gr = gr.drop(index="akveo/ngx-admin")
gr = gr.drop(index="apache/dubbo")
gr = gr.drop(index="facebook/react-native")

In [ ]:
gr.to_excel("pareado.xlsx")

In [ ]:
gr.shape

In [ ]:
a = gr.xs(True, level=1)["maintenance_secs"]

In [ ]:
b = gr.xs(False, level=1)["maintenance_secs"]

In [ ]:
wilcoxon(a,b)

## Análise de todas as releases

In [ ]:
a = f_releases[f_releases["is_last_minor"] == True]["maintenance_secs"]

In [ ]:
b = f_releases[f_releases["is_last_minor"] == False]["maintenance_secs"]

In [ ]:
ranksums(a,b)

## Análise intra-projetos

In [ ]:
intr_project_ds = pd.DataFrame()
intr_project_ds["project"] = f_releases["project"].unique()
intr_project_ds["n_feature_release"] = intr_project_ds["project"].apply(lambda project_name: len(f_releases[f_releases["project"] == project_name]))
intr_project_ds["n_last_minor"] = intr_project_ds["project"].apply(lambda project_name: len(f_releases[(f_releases["project"] == project_name) & (f_releases["is_last_minor"] == True)]))
intr_project_ds = intr_project_ds[intr_project_ds.n_last_minor > 0]
intr_project_ds["p-value"] = intr_project_ds["project"].apply(lambda project_name: ranksums(
    f_releases[(f_releases["is_last_minor"] == True) & (f_releases["project"] == project_name)]["maintenance_secs"],
    f_releases[(f_releases["is_last_minor"] == False) & (f_releases["project"] == project_name)]["maintenance_secs"]
)[1])
intr_project_ds["h0"] = intr_project_ds["p-value"] >= 0.05
intr_project_ds.to_excel("intr_project_ds.xlsx")
intr_project_ds

In [ ]:
project_ds.head()

In [ ]:
project_ds = f_releases.groupby(["project"])["project"]
a

In [ ]:
f_releases.release.count() + f_releases.num_patches.sum() + f_releases.num_pre_releases.sum()

In [ ]:
gr.to_csv("gr_releases.csv", sep=";")

In [ ]:
f_releases.to_csv("releases.csv", sep=";")

In [ ]:
f_releases[f_releases.is_last_minor == False]

In [ ]:
for project in projects.values():
    feature_releases = project.get_releases(skip_patches=True, skip_pre=True)
    print(f"{project.name:20}")
    print(f"{'release':10} {'time':28} {'length':20} {'last_patch':10} {'maintenance':20}")
    for release in feature_releases:
        if release.name.startswith("ng") or release.name.startswith("zone"):
            continue
        if release.patches:
            last_patch = release.patches[-1]
            maintenance = last_patch.time - release.time
        else:
            maintenance = None
            last_patch = None
        if release.pre_releases:
            first_pre = release.pre_releases[0]
        else:
            first_pre = None
          
        print(f"{release.name:10} {str(release.time):28} {str(release.length):20} {str(first_pre):15} {str(last_patch):10} {str(maintenance):20}")

In [ ]:
tags = project.tags
nom_release_tags = [tag for tag in project.tags if not tag.release or tag.name.startswith("zone") or tag.name.startswith("ngcontainer")]
percent_releases = round(100*(1-len(nom_release_tags)/len(tags)),2)
percent_releases

In [ ]:
nom_release_tags

In [ ]:
feature_releases = project.get_releases(skip_patches=True, skip_pre=True)
print(f"{'release':10} {'time':28} {'length':20} {'last_patch':10} {'maintenance':20}")
for release in feature_releases:
    if release.name.startswith("ng") or release.name.startswith("zone"):
        continue
    if release.patches:
        last_patch = release.patches[-1]
        maintenance = last_patch.time - release.time
    else:
        last_patch = None
    print(f"{release.name:10} {str(release.time):28} {str(release.length):20} {str(last_patch):10} {str(maintenance):20}")

In [ ]:
release.patches[-1]

In [ ]:
releases = project.get_releases(skip_pre=True)
for release in releases:
    if release.is_patch():
        print(f"{release.name:20} {str(release.time):30} {str(release.length):10}")